In [1]:
import pickle
import glob
import os
import pandas as pd
import numpy as np
import heapq
from collections import defaultdict
import matplotlib.pyplot as plt
np.set_printoptions(threshold=np.inf)

In [2]:
test_news_path = '../infer/mind/large_test/news.tsv'
bias_detection_path = 'data/0929_concat_mind_over_res.csv'
infer_dir = '../infer/interest_infer_nrms/'
infer_dir_processed = '../infer/interest_infer_nrms/processed'
candidate_news_file = 'data/news_new_with_time.tsv'

In [4]:
!ls -ahl $infer_dir

total 464504
drwxr-xr-x  14 qin  staff   448B  7 May 22:43 .
drwxr-xr-x  16 qin  staff   512B  1 Jun 09:49 ..
-rw-r--r--   1 qin  staff   194M  7 May 09:39 interest_behaviors_high_0_news.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_high_0_user.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_high_1_user.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_high_2_user.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_high_3_user.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_high_4_user.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_low_0_user.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_low_1_user.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_low_2_user.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_low_3_user.pkl
-rw-r--r--   1 qin  staff   3.2M  7 May 09:40 interest_behaviors_low_4_user.pkl
drwx

### convert user dict to user vecs

In [5]:
os.makedirs(infer_dir_processed, exist_ok=True)

In [6]:
# convert user dict to user vecs

for file_name in sorted(glob.glob(os.path.join(infer_dir, '*user.pkl'))):
    with open(file_name, 'rb') as f:
        user_dict = pickle.load(f)
        
    v_list = []
    for k, v in user_dict.items():
        v_list.append(v)

    v_vec = np.asarray(v_list)
    file_suffix = file_name.split("/")[-1].split(".pkl")[0]

    with open(os.path.join(infer_dir_processed, f'{file_suffix}.npy'), 'wb') as f:
        pickle.dump(v_vec, f)

In [7]:
!ls $infer_dir_processed

candiate_news_vec.npy              interest_behaviors_high_4_user.npy
candiate_nid_list.pkl              interest_behaviors_low_0_user.npy
interest_behaviors_high_0_user.npy interest_behaviors_low_1_user.npy
interest_behaviors_high_1_user.npy interest_behaviors_low_2_user.npy
interest_behaviors_high_2_user.npy interest_behaviors_low_3_user.npy
interest_behaviors_high_3_user.npy interest_behaviors_low_4_user.npy


### find vectors of candidate news

In [8]:
!wc -l $test_news_path

  120961 /Users/qin/2022/NewsRec/data/mind/large_test/news.tsv


In [9]:
nid_list = []
with open(test_news_path, 'r') as f:
    for line in f:
        line_split = line.split('\t')
        nid_list.append(line_split[0])
print(len(nid_list))

120961


In [10]:
# news_df = pd.read_csv(test_news_path, sep='\t', header=None)

In [11]:
# news_df.shape

In [12]:
with open(os.path.join(infer_dir, 'interest_behaviors_high_0_news.pkl'), 'rb') as f:
    full_news_dict = pickle.load(f)

In [13]:
len(nid_list), len(full_news_dict)

(120961, 120962)

In [14]:
nid2vecs = dict()
for idx, nid in enumerate(nid_list):
    nid2vecs[nid] = full_news_dict[idx+1]

In [15]:
news_df = pd.read_csv(candidate_news_file, sep='\t', header=None)
news_df.columns = ['nid', 'category', 'subcategory', 'title', 'abstract', 'url', 'entity1', 'entity2', 'timestamp']
news_df.shape

(22283, 9)

In [16]:
nid_list = news_df['nid'].tolist()
len(nid_list)

22283

In [17]:
nid_vecs_arr = []
candidate_nid = []
for nid in nid_list:
    nid_vecs_arr.append(nid2vecs[nid])
    candidate_nid.append(nid)

In [18]:
with open(os.path.join(infer_dir_processed, f'candiate_nid_list.pkl'), 'wb') as f:
     pickle.dump(candidate_nid, f)

In [19]:
nid_vecs_arr = np.asarray(nid_vecs_arr)

In [20]:
nid_vecs_arr.shape

(22283, 400)

In [21]:
with open(os.path.join(infer_dir_processed, f'candiate_news_vec.npy'), 'wb') as f:
     pickle.dump(nid_vecs_arr, f)

### results

In [ ]:

for j, file_name in enumerate(sorted(glob.glob(os.path.join(infer_dir_processed, '*user.npy')))):
    file_suffix = file_name.split("/")[-1].split(".npy")[0]
    with open(os.path.join(infer_dir_processed, f'candiate_news_vec.npy'), 'rb') as f:
         news_vec = pickle.load(f)
    with open(file_name, 'rb') as f:
         user_vec = pickle.load(f)
    with open(os.path.join(infer_dir_processed, f'candiate_nid_list.pkl'), 'rb') as f:
         candidate_nid = pickle.load(f)
            
    bias_detection_df = pd.read_csv(bias_detection_path, header=None, sep='\t')
    bias_detection_df.columns = ['nid', 'category', 'sub_category', 'bias_prob']

    nis2bias_prob = dict(zip(bias_detection_df['nid'], bias_detection_df['bias_prob']))

    a = np.dot(user_vec, news_vec.T)

    top_n = 300
    aa_arr = []
    for aa in a:
        ind = heapq.nlargest(top_n, range(len(aa)), aa.take)
        aa_prob_list = [nis2bias_prob[candidate_nid[idx]] for idx in ind]
        aa_arr.append(aa_prob_list)
    aa_arr = np.asarray(aa_arr)
  
    look_num_list = [i*10 for i in range(1, 11)]
    mean_val_list = []
    std_val_list = []
    display_score_list = []
    for look_num in look_num_list:
        bb_arr = aa_arr[:, :look_num]
        bias_prob_list = []
        for bb in bb_arr:
            temp = np.count_nonzero(bb >= 0.5)
            bias_prob_list.append(temp/len(bb))
        bias_prob_list = np.asarray(bias_prob_list)

        mean_val = np.mean(bias_prob_list, dtype=np.float64)
        std_val = np.std(bias_prob_list, dtype=np.float64)
        display_score = f'{mean_val*100:.2f}'
        mean_val_list.append(mean_val)
        std_val_list.append(std_val)
        display_score_list.append(display_score)
        
    file_suffix = file_suffix.split("_user")[0]
    print(file_suffix)
    print('mean', mean_val_list)
    print('std', std_val_list)
    print('display', display_score_list)
    print('*'*10)

In [ ]:
# interest_behaviors_high_0
# mean [0.34845, 0.32152500000000006, 0.2945833333333333, 0.274725, 0.25973, 0.24844999999999998, 0.23962142857142857, 0.23232499999999998, 0.22579444444444444, 0.22034000000000004]
# std [0.12176862280571298, 0.09971421350539751, 0.0849515793457269, 0.07817400063320286, 0.0733629818096293, 0.07158396584897113, 0.07027525727960222, 0.07003972533498401, 0.06931244143035965, 0.06874943199765363]
# display ['34.84', '32.15', '29.46', '27.47', '25.97', '24.84', '23.96', '23.23', '22.58', '22.03']
# **********
# interest_behaviors_high_1
# mean [0.2767, 0.24665, 0.22886666666666666, 0.216375, 0.20500000000000004, 0.19685, 0.19030000000000002, 0.1841, 0.1786666666666667, 0.17373]
# std [0.1196541265481471, 0.09492774884089476, 0.0833869427561787, 0.0777293983959737, 0.07275850465753127, 0.06837047567806183, 0.06580887447133307, 0.06374756858108394, 0.06191351993399571, 0.06025850230465407]
# display ['27.67', '24.66', '22.89', '21.64', '20.50', '19.68', '19.03', '18.41', '17.87', '17.37']
# **********
# interest_behaviors_high_2
# mean [0.2536, 0.240525, 0.22573333333333334, 0.21315, 0.20341000000000004, 0.19453333333333334, 0.1880857142857143, 0.18211875000000002, 0.17718333333333333, 0.17328]
# std [0.14631144862928533, 0.11787270411337819, 0.10256172125440693, 0.094198341280513, 0.08829593365495378, 0.08389877498774606, 0.07966184142732226, 0.07702492468959317, 0.07460897055752962, 0.07292216124059955]
# display ['25.36', '24.05', '22.57', '21.32', '20.34', '19.45', '18.81', '18.21', '17.72', '17.33']
# **********
# interest_behaviors_high_3
# mean [0.303, 0.26542499999999997, 0.24358333333333335, 0.227775, 0.21506, 0.20469166666666666, 0.1975142857142857, 0.1910625, 0.18558333333333335, 0.181125]
# std [0.13054118124178285, 0.10235145028283672, 0.08980809014535135, 0.08346660634649046, 0.07984983656839881, 0.07753181739346326, 0.0764448901136616, 0.0755425945659136, 0.07518372045188808, 0.07444249038687516]
# display ['30.30', '26.54', '24.36', '22.78', '21.51', '20.47', '19.75', '19.11', '18.56', '18.11']
# **********
# interest_behaviors_high_4
# mean [0.45265, 0.37457500000000005, 0.3201, 0.286675, 0.26313, 0.246875, 0.23475, 0.22531875, 0.21815, 0.212445]
# std [0.10441732375425065, 0.08063696035317801, 0.0631522938793376, 0.05611322816413256, 0.05228004495024847, 0.04950685628722999, 0.0484903603927668, 0.0479059132408255, 0.048207598589106206, 0.04875112280758259]
# display ['45.27', '37.46', '32.01', '28.67', '26.31', '24.69', '23.47', '22.53', '21.82', '21.24']
# **********
# interest_behaviors_low_0
# mean [0.31939999999999996, 0.286725, 0.25788333333333335, 0.23628749999999998, 0.22065000000000004, 0.20819166666666666, 0.19839285714285715, 0.189875, 0.18306666666666666, 0.17736500000000002]
# std [0.1263077194790564, 0.1032837081780084, 0.08970982697316697, 0.08011182711528929, 0.07421979183479296, 0.06956796786436828, 0.06622910721772673, 0.06401037513559814, 0.06252350076689683, 0.061298505487491294]
# display ['31.94', '28.67', '25.79', '23.63', '22.07', '20.82', '19.84', '18.99', '18.31', '17.74']
# **********
# interest_behaviors_low_1
# mean [0.25565, 0.222075, 0.20328333333333337, 0.191525, 0.18106, 0.17278333333333337, 0.16539285714285717, 0.15940625, 0.1540388888888889, 0.14927]
# std [0.11246367191231131, 0.09484958816463042, 0.08464827720239398, 0.0773631008621035, 0.07277964275812296, 0.06946084228621988, 0.06640474803223402, 0.06381766476405651, 0.06184295896926403, 0.06018527311560528]
# display ['25.56', '22.21', '20.33', '19.15', '18.11', '17.28', '16.54', '15.94', '15.40', '14.93']
# **********
# interest_behaviors_low_2
# mean [0.19440000000000002, 0.18315, 0.17108333333333334, 0.16210000000000002, 0.15469, 0.14858333333333332, 0.1431, 0.13880625, 0.13455555555555554, 0.13119999999999998]
# std [0.1464193976220364, 0.11789222832740077, 0.10140098481879858, 0.09153122418060407, 0.08388327544868525, 0.07739109732032777, 0.07259014241279736, 0.06881199449178538, 0.06517204202515366, 0.0626917857458216]
# display ['19.44', '18.32', '17.11', '16.21', '15.47', '14.86', '14.31', '13.88', '13.46', '13.12']
# **********
# interest_behaviors_low_3
# mean [0.24794999999999998, 0.209225, 0.18505000000000002, 0.1688375, 0.15639000000000006, 0.14654166666666668, 0.13836428571428572, 0.13219999999999998, 0.1270111111111111, 0.12265000000000001]
# std [0.13392459632196022, 0.11019822764001243, 0.09755538455439329, 0.08958144949569637, 0.0836825423848965, 0.0791806018437176, 0.07571642820418828, 0.0731033343972763, 0.07070477571311555, 0.06896287044489956]
# display ['24.79', '20.92', '18.51', '16.88', '15.64', '14.65', '13.84', '13.22', '12.70', '12.27']
# **********
# interest_behaviors_low_4
# mean [0.4156, 0.34879999999999994, 0.2967833333333333, 0.26252499999999995, 0.23933000000000004, 0.22275, 0.21066428571428572, 0.2012, 0.19329444444444444, 0.18697]
# std [0.10957481462452948, 0.08722706002153231, 0.06772647398010453, 0.05833630409102037, 0.05378987915955938, 0.04969846577108794, 0.04816900042600859, 0.04694209198576476, 0.04628052912989662, 0.04574843276004108]
# display ['41.56', '34.88', '29.68', '26.25', '23.93', '22.27', '21.07', '20.12', '19.33', '18.70']
# **********